# Input and output formats

This section provides an overview of the input and output formats
supported by DFTK, usually via integration with a third-party library.

## Reading / writing files supported by AtomsIO
[AtomsIO](https://github.com/mfherbst/AtomsIO.jl) is a Julia package which supports
reading / writing atomistic structures from / to a large range of file formats.
Supported formats include Crystallographic Information Framework (CIF),
XYZ and extxyz files, ASE / Gromacs / LAMMPS / Amber trajectory files
or input files of various other codes (e.g. Quantum Espresso, VASP, ABINIT, CASTEP, …).
The full list of formats is is available in the
[AtomsIO documentation](https://mfherbst.github.io/AtomsIO.jl/stable).

The AtomsIO functionality is split into two packages. The main package, `AtomsIO` itself,
only depends on packages, which are registered in the Julia General package registry.
In contrast `AtomsIOPython` extends `AtomsIO` by parsers depending on python packages,
which are automatically managed via `PythonCall`. While it thus provides the full set of
supported IO formats, this also adds additional practical complications, so some users
may choose not to use `AtomsIOPython`.

As an example we start the calculation of a simple antiferromagnetic iron crystal
using a Quantum-Espresso input file, [Fe_afm.pwi](Fe_afm.pwi).
For more details about calculations on magnetic systems
using collinear spin, see Collinear spin and magnetic systems.

First we parse the Quantum Espresso input file using AtomsIO,
which reads the lattice, atomic positions and initial magnetisation
from the input file and returns it as an
[AtomsBase](https://github.com/JuliaMolSim/AtomsBase.jl) `AbstractSystem`,
the JuliaMolSim community standard for representing atomic systems.

In [1]:
using AtomsIO        # Use Julia-only IO parsers
using AtomsIOPython  # Use python-based IO parsers (e.g. ASE)
system = load_system("Fe_afm.pwi")

FlexibleSystem(Fe₂, periodicity = TTT):
    cell_vectors      : [ 2.86814        0        0;
                                0  2.86814        0;
                                0        0  2.86814]u"Å"

    Atom(Fe, [       0,        0,        0]u"Å")
    Atom(Fe, [ 1.43407,  1.43407,  1.43407]u"Å")


Next we build a model making use of the AtomsBase integration
of DFTK and supplying a pseudpotential family
from the [PseudoPotentialData](https://github.com/JuliaMolSim/PseudoPotentialData.jl)

In [2]:
using DFTK
using PseudoPotentialData
pd_lda_family = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
model = model_DFT(system;
                  functionals=LDA(),
                  temperature=0.01,
                  pseudopotentials=pd_lda_family)

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [5.42      , 0         , 0         ]
                           [0         , 5.42      , 0         ]
                           [0         , 0         , 5.42      ]
    unit cell volume     : 159.22 Bohr³

    atoms                : Fe₂
    pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")

    num. electrons       : 32
    spin polarization    : collinear
    temperature          : 0.01 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                           Entropy()

Finally we run the calculation:

In [3]:
basis = PlaneWaveBasis(model; Ecut=10, kgrid=(2, 2, 2))
ρ0 = guess_density(basis, system)
scfres = self_consistent_field(basis, ρ=ρ0);

┌ Warning: Negative ρcore detected: -0.010160696831625918
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/terms/xc.jl:39
n     Energy            log10(ΔE)   log10(Δρ)   Magnet   |Magn|   Diag   Δtime 
---   ---------------   ---------   ---------   ------   ------   ----   ------
  1   -230.6869649508                    0.31   -0.518    0.518    5.2    159ms
  2   -234.1254147329        0.54       -0.32   -0.455    0.455    2.5    193ms
  3   -234.2144302900       -1.05       -1.06   -0.444    0.444    3.5    161ms
  4   -234.2154853332       -2.98       -1.94   -0.447    0.447    2.0    114ms
  5   -234.2155204518       -4.45       -2.77   -0.450    0.450    1.0    103ms
  6   -234.2155223983       -5.71       -2.94   -0.451    0.451    4.0    157ms
  7   -234.2155232470       -6.07       -3.07   -0.453    0.453    1.1    101ms
  8   -234.2155234534       -6.69       -3.13   -0.453    0.453    1.0   98.9ms
  9   -234.2155236469       -6.71       -3.21   -0.454    0.455    1.0   98.6ms
 10   -234.

> **DFTK data formats are not yet fully matured**
>
> The data format in which DFTK saves data as well as the general interface
> of the `load_scfres` and `save_scfres` pair of functions
> are not yet fully matured. They have become fairly stable, but we reserve
> the right to change them incompatibly between DFTK versions (including
> patch versions) at this point.

## Writing VTK files for visualization
For visualizing the density or the Kohn-Sham orbitals DFTK supports storing
the result of an SCF calculations in the form of VTK files.
These can afterwards be visualized using tools such
as [paraview](https://www.paraview.org/).
Using this feature requires
the [WriteVTK.jl](https://github.com/jipolanco/WriteVTK.jl/) Julia package.

In [4]:
using WriteVTK
save_scfres("iron_afm.vts", scfres; save_ψ=true);

This will save the iron calculation above into the file `iron_afm.vts`,
using `save_ψ=true` to also include the KS orbitals.

## Parsable data-export using json
Many structures in DFTK support the (unexported) `todict` function,
which returns a simplified dictionary representation of the data.

In [5]:
DFTK.todict(scfres.energies)

Dict{String, Float64} with 9 entries:
  "AtomicNonlocal" => 0.658901
  "PspCorrection"  => 6.08846
  "Ewald"          => -171.89
  "total"          => -234.216
  "Entropy"        => -0.0383974
  "Kinetic"        => 79.7534
  "AtomicLocal"    => -154.422
  "Hartree"        => 36.1749
  "Xc"             => -30.5409

This in turn can be easily written to disk using a JSON library.
Currently we integrate most closely with `JSON3`,
which is thus recommended.

In [6]:
using JSON3
open("iron_afm_energies.json", "w") do io
    JSON3.pretty(io, DFTK.todict(scfres.energies))
end
println(read("iron_afm_energies.json", String))

{
    "AtomicNonlocal": 0.6589008533776628,
    "PspCorrection": 6.088457202681559,
    "Ewald": -171.88999401525265,
    "total": -234.2155246346467,
    "Entropy": -0.038397427410356455,
    "Kinetic": 79.75341017140744,
    "AtomicLocal": -154.42190527900385,
    "Hartree": 36.174935783284354,
    "Xc": -30.54093192373084
}


Once JSON3 is loaded, additionally a convenience function for saving
a summary of `scfres` objects using `save_scfres` is available:

In [7]:
using JSON3
save_scfres("iron_afm.json", scfres)

Similarly a summary of the band data (occupations, eigenvalues, εF, etc.)
for post-processing can be dumped using `save_bands`:

In [8]:
save_bands("iron_afm_scfres.json", scfres)

Notably this function works both for the results obtained
by `self_consistent_field` as well as `compute_bands`:

In [9]:
bands = compute_bands(scfres, kline_density=10)
save_bands("iron_afm_bands.json", bands)

┌ Warning: The provided cell is a supercell: the returned k-path is the standard k-path of the associated primitive cell in the basis of the supercell reciprocal lattice.
│   cell =
│    Spglib.SpglibCell{Float64, Float64, Int64, Float64}
│     lattice:
│       5.41999997388764  0.0  0.0
│       0.0  5.41999997388764  0.0
│       0.0  0.0  5.41999997388764
│     2 atomic positions:
│       0.0  0.0  0.0
│       0.5  0.5  0.5
│     2 atoms:
│       1  1
│     2 magmoms:
│       0.0  0.0
│    
└ @ BrillouinSpglibExt ~/.julia/packages/Brillouin/DvsT1/ext/BrillouinSpglibExt.jl:28
┌ Warning: Negative ρcore detected: -0.010160696831625918
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/terms/xc.jl:39


## Writing and reading JLD2 files
The full state of a DFTK self-consistent field calculation can be
stored on disk in form of an [JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file.
This file can be read from other Julia scripts
as well as other external codes supporting the HDF5 file format
(since the JLD2 format is based on HDF5). This includes notably `h5py`
to read DFTK output from python.

In [10]:
using JLD2
save_scfres("iron_afm.jld2", scfres)

Saving such JLD2 files supports some options, such as `save_ψ=false`, which avoids saving
the Bloch waves (much faster and smaller files). Notice that JLD2 files can also be used
with `save_bands`.

Since such JLD2 can also be read by DFTK to start or continue a calculation,
these can also be used for checkpointing or for transferring results
to a different computer.
See Saving SCF results on disk and SCF checkpoints for details.

(Cleanup files generated by this notebook.)

In [11]:
rm.(["iron_afm.vts", "iron_afm.jld2",
     "iron_afm.json", "iron_afm_energies.json", "iron_afm_scfres.json",
     "iron_afm_bands.json"]);